In [8]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from utils import *
from models.scorenetwork import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.ndimage.interpolation import rotate
from itertools import chain
from torchvision import transforms
from torchvision.utils import save_image

from IPython.display import HTML
from IPython.display import clear_output

In [11]:
# initialize random seed
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

cuda = True if torch.cuda.is_available() else False
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.set_default_tensor_type(FloatTensor)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sample_dir = './results/MNIST_DAE'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [12]:
joint_training = True
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

eps = 1e-6
sigma_min = 0.001
sigma_max = 10
n_steps = 10
annealed_step = 100

In [13]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../../Data/MNIST/',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [14]:
vae = VAE(input_size=image_size, h_dim=h_dim, z_dim=z_dim, type='ce').to(device)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

sn = SN_Model(device, n_steps, sigma_min, sigma_max, dim=z_dim, p = 0.3)
sn_optim = torch.optim.Adam(sn.parameters(), lr = 0.005)
dynamic = AnnealedLangevinDynamic(sigma_min, sigma_max, n_steps, annealed_step, sn, device, eps=eps)

joint_optim = torch.optim.Adam(params=chain(vae.parameters(), sn.parameters()))

# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        x = x.to(device).view(-1, image_size)
        if joint_training:
            mu, log_var = vae.encode(x)
            z = vae.reparameterize(mu, log_var)
            x_reconst = vae.decode(z)
            # Compute reconstruction loss and kl divergence
            reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
            kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            loss_sn = sn.loss_fn(z)
            loss = loss_sn + reconst_loss + kl_div

            joint_optim.zero_grad()
            loss.backward()
            joint_optim.step()
        else:
            #============= First Stage: Update VAE ==============#
            # Forward pass
            x_reconst, mu, log_var = vae(x)
            # Compute reconstruction loss and kl divergence
            # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
            reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
            kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

            # Backprop and optimize
            vae_loss = reconst_loss + kl_div
            vae_optimizer.zero_grad()
            vae_loss.backward()
            vae_optimizer.step()

            #============= Second Stage: Update SN ==============#
            mu, log_var = vae.encode(x)
            z = vae.reparameterize(mu, log_var)

            loss_sn = sn.loss_fn(z)
            vae_optimizer.zero_grad()
            sn_optim.zero_grad()
            loss_sn.backward()
            sn_optim.step()

        if (i+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}, Diffuse loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item(), loss_sn.item()))

    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(x.shape[0], z_dim).to(device)
        out = vae.decode(z).view(-1, 1, 28, 28)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = vae(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

        # Save the diffused ima ges
        dynamic = AnnealedLangevinDynamic(sigma_min, sigma_max, n_steps, annealed_step, sn, device, eps=eps)
        sample = dynamic.sampling(x.shape[0], z_dim, only_final=True)
        out = vae.decode(sample).view(-1, 1, 28, 28)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'diffuse-{}.png'.format(epoch+1)))


Epoch[1/15], Step [100/469], Reconst Loss: 22262.2383, KL Div: 1389.5240, Diffuse loss: 1.0002
Epoch[1/15], Step [200/469], Reconst Loss: 18096.0859, KL Div: 1948.9353, Diffuse loss: 0.9919
Epoch[1/15], Step [300/469], Reconst Loss: 15765.6279, KL Div: 2313.4150, Diffuse loss: 0.9061
Epoch[1/15], Step [400/469], Reconst Loss: 14814.1807, KL Div: 2550.2549, Diffuse loss: 0.9350
Epoch[2/15], Step [100/469], Reconst Loss: 13195.8281, KL Div: 2684.5554, Diffuse loss: 0.7573
Epoch[2/15], Step [200/469], Reconst Loss: 12802.8047, KL Div: 2912.6306, Diffuse loss: 0.7771
Epoch[2/15], Step [300/469], Reconst Loss: 12581.1191, KL Div: 2880.2310, Diffuse loss: 0.6526
Epoch[2/15], Step [400/469], Reconst Loss: 12251.8145, KL Div: 2997.8713, Diffuse loss: 0.7156
Epoch[3/15], Step [100/469], Reconst Loss: 11663.4277, KL Div: 3072.2434, Diffuse loss: 0.7507
Epoch[3/15], Step [200/469], Reconst Loss: 11646.7480, KL Div: 3059.5444, Diffuse loss: 0.7011
Epoch[3/15], Step [300/469], Reconst Loss: 11290.7

In [18]:
def ani_imshow(sample, sampling_number = 64):

    row_number = int(math.sqrt(sampling_number))
    col_number = int(math.sqrt(sampling_number))
    sample = sample[:sampling_number].detach().cpu().numpy()
    shape = sample.shape
    show_sample = np.zeros([row_number * shape[2], col_number * shape[3] ]).astype(np.float32)
    for row in range(row_number):
        for col in range(col_number):
            sample_ = sample[row + col * row_number][0]
            show_sample[ row * shape[2] : (row+1) * shape[2], col * shape[3] : (col+1) * shape[3] ] = (sample_ - sample_.min()) / (sample_.max() - sample_.min()) * 255

    show_sample = show_sample.astype(np.uint8)

    return show_sample

In [1]:
only_final = False
n_steps = 8
test_size = 36
dynamic = AnnealedLangevinDynamic(sigma_min, sigma_max, n_steps, annealed_step, sn, device, eps=eps)
sample = dynamic.sampling(test_size, z_dim, only_final)
diffuse_data = vae.decode(sample)

step_size = sample.size(0)
fig, axs = plt.subplots(1, step_size, figsize=(step_size * 6, 6), constrained_layout = True)
for i in range(step_size):
    diff_data = diffuse_data[i].view(-1, 1, 28, 28)
    axs[i].imshow(ani_imshow(diff_data, sampling_number = test_size), animated=True, cmap = 'gray')
    # plt.show()

NameError: name 'AnnealedLangevinDynamic' is not defined